# S_EllipsoidTestMicroprice [<img src="https://www.arpm.co/lab/icons/icon_permalink.png" width=30 height=30 style="display: inline;">](https://www.arpm.co/lab/redirect.php?code=S_EllipsoidTestMicroprice&codeLang=Python)
For details, see [here](https://www.arpm.co/lab/redirect.php?permalink=IIDHFmicroprice).

## Prepare the environment

In [ ]:
import os
import os.path as path
import sys

sys.path.append(path.abspath('../../functions-legacy'))

import numpy as np
from numpy import arange, interp, floor, diff

from scipy.io import loadmat

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.style.use('seaborn')

from CONFIG import GLOBAL_DB, TEMPORARY_DB
from ARPM_utils import save_plot
from autocorrelation import autocorrelation
from InvarianceTestEllipsoid import InvarianceTestEllipsoid

## Load the database generated by the script S_HighFreqVolumeTime

In [ ]:
try:
    db = loadmat(os.path.join(GLOBAL_DB, 'db_HighFreqVolumeTime'), squeeze_me=True)
except FileNotFoundError:
    db = loadmat(os.path.join(TEMPORARY_DB, 'db_HighFreqVolumeTime'),
                 squeeze_me=True)  # generated by S_HighFreqVolumeTime

p_mic = db['p_mic'].reshape(1,-1)
t_k = db['t_k']
t_ms = db['t_ms']
q_t = db['q_t']
q = db['q']

## Compute microprice time series

In [ ]:
delta_t = 2000  # selecting observations every 2 seconds
p_mic_t = p_mic[[0],::delta_t]

delta_q = floor((np.nanmax(q_t) - np.nanmin(q_t)) / p_mic_t.shape[1])  # width of activity time bins
volume_time = arange(np.nanmin(q_t),np.nanmax(q_t)+delta_q,delta_q).reshape(1,-1)  # vector of volume times
t_q = interp(volume_time, q, t_k)  # vector of wall clock time as a function of volume time
p_mic_q = interp(t_q, t_ms, p_mic[0])  # time changed microprice, i.e. microprice as a function of volume time

## Compute microprice increments

In [ ]:
delta_p_mic_t = diff(p_mic_t)
delta_p_mic_q = diff(p_mic_q)

In [ ]:
# ## Compute autocorrelations at different lags
lag_ = 10

In [ ]:
acf_t = autocorrelation(delta_p_mic_t, lag_)
acf_q = autocorrelation(delta_p_mic_q, lag_)

## Plot the results of the IID test

In [ ]:
lag = 10  # lag to be printed
ell_scale = 2  # ellipsoid radius coefficient
fit = 0  # normal fitting

f = figure(figsize=(12,6))  # changes in implied vol
InvarianceTestEllipsoid(delta_p_mic_t, acf_t[0,1:], lag, fit, ell_scale, [], 'IID test on the increments of microprice');
# save_plot(ax=plt.gca(), extension='png', scriptname=os.path.basename('.')[:-3], count=plt.get_fignums()[-1])

f = figure(figsize=(12,6))  # changes in log implied vol
pos = {}
pos[0]= [.2, .45, .3866, .43]  # scatter plot
pos[1]= [.2905, .12, .205, .2157]  # epsi
pos[2]= [.045, .45, .1437, .43]  # epsi_lagged
pos[3]= [.6, .45, .3366, .43]  # autocorrelation
pos[4]= [.085, .228, .11, .1]  # leg
InvarianceTestEllipsoid(delta_p_mic_q,acf_q[0,1:], lag, fit, ell_scale, pos, 'IID test on the increments of time-changed microprice');
# save_plot(ax=plt.gca(), extension='png', scriptname=os.path.basename('.')[:-3], count=plt.get_fignums()[-1])